# Ridge and Lassso Regression

[Harvard Example Resource](https://harvard-iacs.github.io/2017-CS109A/lectures/lecture7/notebook/)

**Note:** After a decent amount of searching, I can't seem to find the data associated with the Harvard resource. I'm gonna leave it here though, as it could certainly come in handy later.

We'll use [this resource](https://refactored.ai/microcourse/notebook?path=content/05-Regression_models_in_Machine_Learning/05-Regularization/03-Lab-Regularization_guided.ipynb) instead, but either way both seem to use `sklearn` for the ridge and lasso. Given this, might as well just follow along with the example to get some practice.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
import numpy as np
import matplotlib.pyplot as plt